![rmotr](https://user-images.githubusercontent.com/7065401/52071918-bda15380-2562-11e9-828c-7f95297e4a82.png)
<hr style="margin-bottom: 40px;">

<img src="https://user-images.githubusercontent.com/7065401/68501079-0695df00-023c-11ea-841f-455dac84a089.jpg"
    style="width:400px; float: right; margin: 0 40px 40px 40px;"></img>

# Reading JSON files

In this lecture we'll learn how to read JSON files (.json) into a pandas `DataFrame`, and how to export that `DataFrame` to a JSON file.

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Hands on! 

In [ ]:
import pandas as pd

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## The `read_json` method

We'll begin with the **read_json** method, that let us read simple JSON files into a `DataFrame`.

This `read_json` method accepts many parameters as we saw on `read_csv` and `read_excel`, such as `filepath`, `dtype` and `encoding`.

> Full `read_json` documentation can be found here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html.

In this case we'll try to read our `games.json` JSON file.

This file contains records of PlayStation games in Europe with its title, price, provider and genre.

In [ ]:
!head -n20 games.json

In [ ]:
games = pd.read_json('games.json')

In [ ]:
games.head()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Nested JSON example

JSON documents tend not to be so straightforward to read as tables, specially when they have nested structures.

[Dataset source](https://jsonplaceholder.typicode.com/users)

In [ ]:
!head -n20 users.json

In [ ]:
df = pd.read_json('users.json')

In [ ]:
df.head()

This is not the data structure we wanted, so in this cases `read_json` could not be the best solution.

We'll need to use the `json` Python module to parse our JSON file into a Python Dictionary object, to be able to index that dictionary and select nested data we want.

To do that we'll use the `json.load()` method, that will parse our JSON file into a Python Dictionary `json_dict`.

In [ ]:
import json

with open('users.json') as file:
    json_dict = json.load(file)

In [ ]:
json_dict

In [ ]:
json_dict.keys()

In [ ]:
json_dict.values()

In [ ]:
for user in json_dict['info']:
    print(user)

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Using pandas `from_dict` method

With our Python Dictionary ready, we'll introduce another useful pandas method: `from_dict()`.

This `from_dict` method will construct a new `DataFrame` from a dict of array-like or dicts.

> Full `from_dict` documentation can be found here: https://pandas.pydata.org/pandas-docs/version/0.25/reference/api/pandas.DataFrame.from_dict.html

In [ ]:
pd.DataFrame.from_dict({'Fruits': ['Apple', 'Banana']})

### Changing data orientation using `orient` parameter

The "orientation" of the data. If the keys of the passed dictionary should be the columns of the resulting `DataFrame`, pass `orient=columns` (default behaviour). Otherwise if the keys should be rows, use `orient=index`.

In [ ]:
pd.DataFrame.from_dict({'Fruits': ['Apple', 'Banana']}, orient='index')

Go ahead and use our `json_dict` dictionary to create a new `DataFrame`, but this time using the `value` attribute:

In [ ]:
df = pd.DataFrame.from_dict(json_dict)

In [ ]:
df.head()

In [ ]:
df = pd.DataFrame.from_dict(json_dict['info'])

In [ ]:
df.head()

This way our data looks more understandable. Each cell has a correct data structure.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Using `json_normalize` for complex nesting

In our example we still have two more complex columns, `address` and `company`.

- `address`: list of dictionaries (records)
- `company`: dictionary (record)

The `json_normalize` method will be useful to unpack and flatten that data easily.

Let's unpack the works column into a standalone dataframe. We'll also grab the flat columns so we can do analysis.

In [ ]:
df.head()

In [ ]:
json_dict

In [ ]:
from pandas.io.json import json_normalize

users = json_normalize(json_dict)

users

In [ ]:
users = json_normalize(json_dict['info'])

In [ ]:
users.head(3)

Now the `company` column was exploted in three new columns: `company.name`, `company.catchPhrase` and `company.bs`. 

### Custom separator for new columns

We can use the `sep` parameter to define how will nested records will be separated by.

In [ ]:
users = json_normalize(json_dict['info'],
                       sep='_')

In [ ]:
users.head(3)

### Unpack column with a list of records

When we have a column with a list of nested values (records, dictionary) it's difficult to deal with it.

`json_normalize` let us define a `record_path` parameter to define the path to a column with a list of records to unpack them. If this parameter is not passed, data will be assumed to be an array of records.

In our case we'll unpack the `address` column into a standalone `DataFrame`.

In [ ]:
address = json_normalize(json_dict['info'],
                         sep='_',
                         record_path='address')

In [ ]:
address.head()

The `record_path` parameter could receive a list of columns indicating the nested path.

In [ ]:
address = json_normalize(json_dict,
                         sep='_',
                         record_path=['info', 'address'])

In [ ]:
address.head()

### Adding extra columns to unpacked columns

There is another useful parameter, `meta`, which allow us to add fields as metadata for each record in the resulting `DataFrame`.

In our case we'll add the user identifier to each address.

In [ ]:
address = json_normalize(json_dict['info'],
                         sep='_',
                         record_path='address',
                         meta=['id'])

In [ ]:
address.head()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Save to JSON file

Finally we can save our `DataFrame` as a JSON file.

In [ ]:
users.head()

We can simply generate a JSON string from our `DataFrame`:

In [ ]:
users.to_json()

Or specify a file path where we want our generated JSON code to be saved:

In [ ]:
users.to_json('out.json')

In [ ]:
pd.read_json('out.json').head()

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)